# Hate Speech on Twitter
## Deliverable 03
## Amir ElTabakh
## 3/18/2022

This document primarily focuses on accessing the Twitter API, the privileges we have to work with, extended privileges for when we are ready to move forward with our analysis, how we can query Twitter searches, and identifying the features we have to work with.

### Agenda
- Collect 10,000 tweets from the twitter API
- Clean the dataset
- Perform Sentiment Analysis on the tweets
- Generate a map with the data

We now have access to the Elevated product. I will be applying for the Academic Research Product, hopefully I am approved for the privilege sooner rather than later.

## Collect Tweets

In [1]:
# pip install tweepy (use either or)
#!pip install tweep
!python -m pip install git+https://github.com/tweepy/tweepy@master

  Cloning https://github.com/tweepy/tweepy (to revision master) to c:\users\amira\appdata\local\temp\pip-req-build-f4yaaz89


  Running command git clone -q https://github.com/tweepy/tweepy 'C:\Users\amira\AppData\Local\Temp\pip-req-build-f4yaaz89'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/tweepy/tweepy 'C:\Users\amira\AppData\Local\Temp\pip-req-build-f4yaaz89'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [2]:
# importing dependencies
import tweepy as tw
import pandas as pd

# Twitter Keys, tokens, and secrets are saved in seperate config file on my local device
from config import consumer_key, consumer_secret, access_token, access_secret, bearer_token

# authenticate
auth = tw.OAuthHandler(consumer_key, consumer_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Here we set up our search_query to fetch tweets with `#chinesevirus` but also filter out the retweets. You can customize your query based on your requirements. For more, refer to [this guide](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/guides/standard-operators).

We can query exact phrases, we could add an 'or' clause, a 'minus' clause to query strings not containing a string, we can query hashtags, tweets sent from specific twitter accounts, a tweet authored tin reply to an account, tweets mentioning accounts, we can filter for or against tweets marked as potentially sensitive, tweets containing images and/or videos, tweets containing URLs (you can filter for strings inside the URL as well), you can filter for tweets sent before a date or since a date, you can filter for tweets containing positive attitudes or negative attitudes, and you can filter for tweets containing questions.

^ A lot I know, but these may come in handy when figuring out how we want to target our tweets.

Now that we can access the API, Let's get our tweets.

In [3]:
# get tweets from the API
search_query = "#chinesevirus -filter:retweets"

# center of united states
latitude = "37.09024"
longitude = "-95.712891"
radius = "791mi"
location = f"{latitude},{longitude},{radius}"

tweets = tw.Cursor(api.search_tweets,
                  q = search_query,
                  lang = "en",
                  geocode = location).items(5)

# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    print(tweet.text, '\n\n')
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

#ChineseVirus spreading like wildfire in China ...Ms. KARMA just flexing. https://t.co/3rpvagAZW4 


@sophielouisecc #COVID19 #omicron #chinesevirus #Pfizer _ Pfizer doesn't want the covid19 tap to be turned off. 


@WelshGovernment #COVID19 #Omicron #chinesevirus _ Yet again, the Welsh Government issues a tweet that can't be rep… https://t.co/5WZsys1vQG 


@DrEricDing Covid19 #omicron #chinesevirus _ Perhaps they should get the scientists in Wuhan to create a vaccine fo… https://t.co/JdnXFcSzAo 


@Sloganyo @ETSU_MBB Indeed.  That was a great team.  #ChineseVirus 


Total Tweets fetched: 5


In [4]:
# snapshot of how a single tweet looks like
tweets_copy[4]._json

{'created_at': 'Mon Mar 14 09:12:37 +0000 2022',
 'id': 1503298039452258304,
 'id_str': '1503298039452258304',
 'text': '@Sloganyo @ETSU_MBB Indeed.  That was a great team.  #ChineseVirus',
 'truncated': False,
 'entities': {'hashtags': [{'text': 'ChineseVirus', 'indices': [53, 66]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'Sloganyo',
    'name': 'Logan Harris',
    'id': 771576011137781760,
    'id_str': '771576011137781760',
    'indices': [0, 9]},
   {'screen_name': 'ETSU_MBB',
    'name': "ETSU Men's Basketball",
    'id': 2310212678,
    'id_str': '2310212678',
    'indices': [10, 19]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': 1503179772104486922,
 'in_reply_to_status_id_str': '1503179772104486922',
 'in_reply_to_user_id': 771576011137781760,
 'in_reply_to_user_id_str': '771576011137781760',
 'in_reply_to_screen_nam

In [5]:
# Lets create a dataframe for our scraped tweets
df = pd.DataFrame(columns = tweets_copy[0]._json.keys())

for i in range(len(tweets_copy)):
    df = df.append(pd.json_normalize(tweets_copy[i]._json))
    
df.reset_index()
df

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,quoted_status.place,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.lang,user.profile_banner_url
0,Wed Mar 16 10:35:42 +0000 2022,1504043726024462339,1504043726024462339,#ChineseVirus spreading like wildfire in China...,False,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",None,None,...,None,None,False,56.0,88.0,False,False,False,en,NaN
0,Tue Mar 15 19:06:03 +0000 2022,1503809770070937618,1503809770070937618,@sophielouisecc #COVID19 #omicron #chineseviru...,False,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1503776654262419458,1503776654262419458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Tue Mar 15 19:01:14 +0000 2022,1503808560832172036,1503808560832172036,@WelshGovernment #COVID19 #Omicron #chinesevir...,True,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Mon Mar 14 18:11:05 +0000 2022,1503433550292525059,1503433550292525059,@DrEricDing Covid19 #omicron #chinesevirus _ P...,True,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1503270983079280643,1503270983079280643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Mon Mar 14 09:12:37 +0000 2022,1503298039452258304,1503298039452258304,@Sloganyo @ETSU_MBB Indeed. That was a great ...,False,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1503179772104486922,1503179772104486922,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://pbs.twimg.com/profile_banners/38787045...


In [6]:
# all columns available to us
df.columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities',
       'metadata', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str',
       ...
       'quoted_status.place', 'quoted_status.contributors',
       'quoted_status.is_quote_status', 'quoted_status.retweet_count',
       'quoted_status.favorite_count', 'quoted_status.favorited',
       'quoted_status.retweeted', 'quoted_status.possibly_sensitive',
       'quoted_status.lang', 'user.profile_banner_url'],
      dtype='object', length=151)

In [7]:
# Number of columns available to us
len(df.columns)

151

The user has to explicitly share its location while tweeting for tweet.coordinates to be equal to the tweet exact location, so it makes sense many tweets will have empty coordinates. Thankfully for us the `user.location` column is a good workaround, still we may not find a location for all users.

Here are the [GeoGuidelines](https://developer.twitter.com/en/developer-terms/geo-guidelines).

In [8]:
list_of_features_to_keep = ['created_at', 'id', 'id_str', 'text', 'truncated', 'entities.hashtags', 'entities.user_mentions',
                            'source', 'user.location', 'user.id', 'user.id_str', 'user.name',
                            'user.screen_name', 'user.description', 'user.url', 
                            'coordinates', 'place', 'contributors', 'is_quote_status', 'user.statuses_count',
                            'user.followers_count', 'user.friends_count', 'user.listed_count', 
                            'user.created_at', 'user.favourites_count', 'user.utc_offset', 'user.time_zone',
                            'retweet_count', 'favorite_count', 'lang', 'possibly_sensitive'
                            ]
df = df[list_of_features_to_keep]

# reset index
df = df.reset_index()
del df['index']

# output dataframe
df

,created_at,id,id_str,text,truncated,entities.hashtags,entities.user_mentions,source,user.location,user.id,...,user.friends_count,user.listed_count,user.created_at,user.favourites_count,user.utc_offset,user.time_zone,retweet_count,favorite_count,lang,possibly_sensitive
0,Wed Mar 16 10:35:42 +0000 2022,1504043726024462339,1504043726024462339,#ChineseVirus spreading like wildfire in China...,False,"[{'text': 'ChineseVirus', 'indices': [0, 13]}]",[],"<a href=""http://twitter.com/download/android"" ...","Chapin, SC",31394019.0,...,420.0,2.0,Wed Apr 15 12:21:25 +0000 2009,24053.0,None,None,0,0,en,False
1,Tue Mar 15 19:06:03 +0000 2022,1503809770070937618,1503809770070937618,@sophielouisecc #COVID19 #omicron #chineseviru...,False,"[{'text': 'COVID19', 'indices': [16, 24]}, {'t...","[{'screen_name': 'sophielouisecc', 'name': 'So...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,66236946.0,...,71.0,5.0,Mon Aug 17 01:06:33 +0000 2009,11947.0,None,None,0,0,en,NaN
2,Tue Mar 15 19:01:14 +0000 2022,1503808560832172036,1503808560832172036,@WelshGovernment #COVID19 #Omicron #chinesevir...,True,"[{'text': 'COVID19', 'indices': [17, 25]}, {'t...","[{'screen_name': 'WelshGovernment', 'name': 'W...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,66236946.0,...,71.0,5.0,Mon Aug 17 01:06:33 +0000 2009,11947.0,None,None,0,0,en,NaN
3,Mon Mar 14 18:11:05 +0000 2022,1503433550292525059,1503433550292525059,@DrEricDing Covid19 #omicron #chinesevirus _ P...,True,"[{'text': 'omicron', 'indices': [20, 28]}, {'t...","[{'screen_name': 'DrEricDing', 'name': 'Eric F...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,66236946.0,...,71.0,5.0,Mon Aug 17 01:06:33 +0000 2009,11947.0,None,None,0,0,en,NaN
4,Mon Mar 14 09:12:37 +0000 2022,1503298039452258304,1503298039452258304,@Sloganyo @ETSU_MBB Indeed. That was a great ...,False,"[{'text': 'ChineseVirus', 'indices': [53, 66]}]","[{'screen_name': 'Sloganyo', 'name': 'Logan Ha...","<a href=""https://mobile.twitter.com"" rel=""nofo...","Knoxville, TN",38787045.0,...,4984.0,45.0,Sat May 09 03:18:48 +0000 2009,130371.0,None,None,0,0,en,NaN


On another note, the Twitter API only shows the first 140 characters of a tweet. By taking additional steps, we can get a total of 280 characters per tweet. Let's do so for all tweets.

In [9]:
# print tweet text
for i in range(len(df['text'])):
    print(df['text'][i], '\n\n')

#ChineseVirus spreading like wildfire in China ...Ms. KARMA just flexing. https://t.co/3rpvagAZW4 


@sophielouisecc #COVID19 #omicron #chinesevirus #Pfizer _ Pfizer doesn't want the covid19 tap to be turned off. 


@WelshGovernment #COVID19 #Omicron #chinesevirus _ Yet again, the Welsh Government issues a tweet that can't be rep… https://t.co/5WZsys1vQG 


@DrEricDing Covid19 #omicron #chinesevirus _ Perhaps they should get the scientists in Wuhan to create a vaccine fo… https://t.co/JdnXFcSzAo 


@Sloganyo @ETSU_MBB Indeed.  That was a great team.  #ChineseVirus 




In [10]:
for i in range(len(df)):
    # only do this for those tweets that are truncated
    if df.at[i, 'truncated'] == True:
        # print out the text as it is in df
        print(f"Truncated Text: \n{df['text'][i]}\n")
        
        # get ID of the tweet
        id_str = list(df['id_str'])[i]
        
        # verify API status and get tweet info
        status = api.get_status(id_str, tweet_mode="extended")
        
        # replace truncated text with full text
        try:
            print(f"Complete Text: \n{status.retweeted_status.full_text}\n\n")
            df['text'][i] = status.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            print(f"Complete Text: \n{status.full_text}\n\n")
            df['text'][i] = status.full_text
            
        # set the value of truncated at index i to False
        df.at[i, 'truncated'] = False

Truncated Text: 
@WelshGovernment #COVID19 #Omicron #chinesevirus _ Yet again, the Welsh Government issues a tweet that can't be rep… https://t.co/5WZsys1vQG

Complete Text: 
@WelshGovernment #COVID19 #Omicron #chinesevirus _ Yet again, the Welsh Government issues a tweet that can't be replied to.
They're obviously scared of the end of covid19, as it means they'll have to actually start governing.


Truncated Text: 
@DrEricDing Covid19 #omicron #chinesevirus _ Perhaps they should get the scientists in Wuhan to create a vaccine fo… https://t.co/JdnXFcSzAo



C:\Users\amira\AppData\Local\Temp/ipykernel_1432/3232376038.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = status.full_text


Complete Text: 
@DrEricDing Covid19 #omicron #chinesevirus _ Perhaps they should get the scientists in Wuhan to create a vaccine for the covid virus they invented.




In [11]:
# print tweet text
for i in range(len(df['text'])):
    print(df['text'][i], '\n\n')

#ChineseVirus spreading like wildfire in China ...Ms. KARMA just flexing. https://t.co/3rpvagAZW4 


@sophielouisecc #COVID19 #omicron #chinesevirus #Pfizer _ Pfizer doesn't want the covid19 tap to be turned off. 


@WelshGovernment #COVID19 #Omicron #chinesevirus _ Yet again, the Welsh Government issues a tweet that can't be replied to.
They're obviously scared of the end of covid19, as it means they'll have to actually start governing. 


@DrEricDing Covid19 #omicron #chinesevirus _ Perhaps they should get the scientists in Wuhan to create a vaccine for the covid virus they invented. 


@Sloganyo @ETSU_MBB Indeed.  That was a great team.  #ChineseVirus 




Let's clean clean text by using regular expressions. We'll remove any hyperlinks, punctuation characters and finally convert to lowercase.

In [12]:
#library for regular expressions
import re

tweets_cleaned = []

for tweet in df['text']:
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.\S+', "", tweet)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # convert to lower case
    tweet = tweet.lower()
    
    # append tweet text to list of cleaned tweet texts
    tweets_cleaned += [tweet]

# convert list to Pandas Series object
tweets_cleaned_series = pd.Series(tweets_cleaned)

# Add text_cleaned as column to df
df.insert(df.columns.get_loc("text") + 1, "text_cleaned", tweets_cleaned_series)

Let's convert the timestamp from UTC to EST.

In [13]:
from pytz import timezone
from datetime import datetime

def utc_to_est(tweet_timestamp):
        '''
        Utility function to convert timestamps from UTC to EST.
        '''
        
        eastern = timezone('US/Eastern')
        utc = timezone('UTC')
        created_at = datetime.strptime(tweet_timestamp, '%a %b %d %H:%M:%S +0000 %Y')
        utc_created_at = utc.localize(created_at)
        est_created_at = utc_created_at.astimezone(eastern)
        
        return est_created_at
    
# tweet created at
for i in range(len(df)):
    print(df['created_at'][i])
    print(utc_to_est(df['created_at'][i]))
    df['created_at'][i] = utc_to_est(df['created_at'][i])
    
# user account created at
for i in range(len(df)):
    print(df['user.created_at'][i])
    print(utc_to_est(df['user.created_at'][i]))
    df['user.created_at'][i] = utc_to_est(df['user.created_at'][i])

Wed Mar 16 10:35:42 +0000 2022
2022-03-16 06:35:42-04:00
Tue Mar 15 19:06:03 +0000 2022
2022-03-15 15:06:03-04:00
Tue Mar 15 19:01:14 +0000 2022
2022-03-15 15:01:14-04:00
Mon Mar 14 18:11:05 +0000 2022
2022-03-14 14:11:05-04:00
Mon Mar 14 09:12:37 +0000 2022
2022-03-14 05:12:37-04:00
Wed Apr 15 12:21:25 +0000 2009
2009-04-15 08:21:25-04:00
Mon Aug 17 01:06:33 +0000 2009
2009-08-16 21:06:33-04:00
Mon Aug 17 01:06:33 +0000 2009
2009-08-16 21:06:33-04:00
Mon Aug 17 01:06:33 +0000 2009
2009-08-16 21:06:33-04:00
Sat May 09 03:18:48 +0000 2009
2009-05-08 23:18:48-04:00


C:\Users\amira\AppData\Local\Temp/ipykernel_1432/2832659699.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'][i] = utc_to_est(df['created_at'][i])
C:\Users\amira\AppData\Local\Temp/ipykernel_1432/2832659699.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user.created_at'][i] = utc_to_est(df['user.created_at'][i])


In [14]:
# lets take a look at the dataframe now
df

,created_at,id,id_str,text,text_cleaned,truncated,entities.hashtags,entities.user_mentions,source,user.location,...,user.friends_count,user.listed_count,user.created_at,user.favourites_count,user.utc_offset,user.time_zone,retweet_count,favorite_count,lang,possibly_sensitive
0,2022-03-16 06:35:42-04:00,1504043726024462339,1504043726024462339,#ChineseVirus spreading like wildfire in China...,chinesevirus spreading like wildfire in china ...,False,"[{'text': 'ChineseVirus', 'indices': [0, 13]}]",[],"<a href=""http://twitter.com/download/android"" ...","Chapin, SC",...,420.0,2.0,2009-04-15 08:21:25-04:00,24053.0,None,None,0,0,en,False
1,2022-03-15 15:06:03-04:00,1503809770070937618,1503809770070937618,@sophielouisecc #COVID19 #omicron #chineseviru...,@sophielouisecc covid19 omicron chinesevirus p...,False,"[{'text': 'COVID19', 'indices': [16, 24]}, {'t...","[{'screen_name': 'sophielouisecc', 'name': 'So...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,...,71.0,5.0,2009-08-16 21:06:33-04:00,11947.0,None,None,0,0,en,NaN
2,2022-03-15 15:01:14-04:00,1503808560832172036,1503808560832172036,@WelshGovernment #COVID19 #Omicron #chinesevir...,@welshgovernment covid19 omicron chinesevirus ...,False,"[{'text': 'COVID19', 'indices': [17, 25]}, {'t...","[{'screen_name': 'WelshGovernment', 'name': 'W...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,...,71.0,5.0,2009-08-16 21:06:33-04:00,11947.0,None,None,0,0,en,NaN
3,2022-03-14 14:11:05-04:00,1503433550292525059,1503433550292525059,@DrEricDing Covid19 #omicron #chinesevirus _ P...,@drericding covid19 omicron chinesevirus _ per...,False,"[{'text': 'omicron', 'indices': [20, 28]}, {'t...","[{'screen_name': 'DrEricDing', 'name': 'Eric F...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,...,71.0,5.0,2009-08-16 21:06:33-04:00,11947.0,None,None,0,0,en,NaN
4,2022-03-14 05:12:37-04:00,1503298039452258304,1503298039452258304,@Sloganyo @ETSU_MBB Indeed. That was a great ...,@sloganyo @etsu_mbb indeed. that was a great ...,False,"[{'text': 'ChineseVirus', 'indices': [53, 66]}]","[{'screen_name': 'Sloganyo', 'name': 'Logan Ha...","<a href=""https://mobile.twitter.com"" rel=""nofo...","Knoxville, TN",...,4984.0,45.0,2009-05-08 23:18:48-04:00,130371.0,None,None,0,0,en,NaN


Now I believe I've cleaned this dataset up well enough to be of use. We can review what else there is to clean and get around to it.

## Sentiment Analysis

In Natural Language Processing there is a concept known as Sentiment Analysis. Given a movie review or a tweet, it can be automatically classified in categories. These categories can be user defined (positive, negative) or whichever classes you want.

I'll be using the [TextBlob](https://textblob.readthedocs.io/en/dev/) library. TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

The `sentiment` score returns a tuple of the form (polarity, subjectivity ) where polarity is a float within the range [-1.0, 1.0] where 0 indicates neutral, +1 indicates a very positive sentiment and -1 represents a very negative sentiment. Subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [19]:
# example
from textblob import TextBlob
data = TextBlob(df['text_cleaned'][4])
print(data)

data.sentiment

@sloganyo @etsu_mbb indeed.  that was a great team.  chinesevirus


Sentiment(polarity=0.8, subjectivity=0.75)

Another approach is to use Vader. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. VADER uses a combination of A sentiment lexicon is a list of lexical features (e.g., words) which are generally labeled according to their semantic orientation as either positive or negative. VADER not only tells about the Positivity and Negativity score but also tells us about how positive or negative a sentiment is.

The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between -1(most extreme negative) and +1 (most extreme positive).
- positive sentiment : (compound score >= 0.05) 
- neutral sentiment : (compound score > -0.05) and (compound score < 0.05) 
- negative sentiment : (compound score <= -0.05)

In [16]:
# install vader onto device
!pip install vaderSentiment

In [20]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

data = TextBlob(df['text_cleaned'][1])
print(data)

SentimentIntensityAnalyzer().polarity_scores(data)

@sophielouisecc covid19 omicron chinesevirus pfizer _ pfizer doesn't want the covid19 tap to be turned off.


{'neg': 0.075, 'neu': 0.925, 'pos': 0.0, 'compound': -0.0572}

In [18]:
# here is another example

# init the sentiment analyzer
sia = SentimentIntensityAnalyzer()

sentences = [
    "This food is amazing and tasty !",
    "Exoplanets are planets outside the solar system",
    "This is sad to see such bad behavior"
            ]

for sentence in sentences:
    score = sia.polarity_scores(sentence)["compound"]
    print(f'The sentiment value of the sentence: "{sentence}" is : {score}')

The sentiment value of the sentence: "This food is amazing and tasty !" is : 0.6239
The sentiment value of the sentence: "Exoplanets are planets outside the solar system" is : 0.0
The sentiment value of the sentence: "This is sad to see such bad behavior" is : -0.765


Now, I can use cleaned text to calculate polarity, subjectivity, sentiment, negative, positive, neutral and compound parameters again. For all calculated parameters, I create new features to my data frame.

In [23]:
# calulcating polarity and subjectivity for all tweets
df[['polarity', 'subjectivity']] = df['text_cleaned'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

for index, row in df['text_cleaned'].iteritems():
    # using Vader to get sentiment scores
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    
    # determine sentiment
    if neg > pos:
        df.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        df.loc[index, 'sentiment'] = "positive"
    else:
        df.loc[index, 'sentiment'] = "neutral"
    
    # populating neg, neutral, positive, and compound columns
    df.loc[index, 'neg'] = neg
    df.loc[index, 'neu'] = neu
    df.loc[index, 'pos'] = pos
    df.loc[index, 'compound'] = comp

df

,created_at,id,id_str,text,text_cleaned,truncated,entities.hashtags,entities.user_mentions,source,user.location,...,favorite_count,lang,possibly_sensitive,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,2022-03-16 06:35:42-04:00,1504043726024462339,1504043726024462339,#ChineseVirus spreading like wildfire in China...,chinesevirus spreading like wildfire in china ...,False,"[{'text': 'ChineseVirus', 'indices': [0, 13]}]",[],"<a href=""http://twitter.com/download/android"" ...","Chapin, SC",...,0,en,False,0.0,0.00,positive,0.000,0.783,0.217,0.3612
1,2022-03-15 15:06:03-04:00,1503809770070937618,1503809770070937618,@sophielouisecc #COVID19 #omicron #chineseviru...,@sophielouisecc covid19 omicron chinesevirus p...,False,"[{'text': 'COVID19', 'indices': [16, 24]}, {'t...","[{'screen_name': 'sophielouisecc', 'name': 'So...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,...,0,en,NaN,0.0,0.00,negative,0.075,0.925,0.000,-0.0572
2,2022-03-15 15:01:14-04:00,1503808560832172036,1503808560832172036,@WelshGovernment #COVID19 #Omicron #chinesevir...,@welshgovernment covid19 omicron chinesevirus ...,False,"[{'text': 'COVID19', 'indices': [17, 25]}, {'t...","[{'screen_name': 'WelshGovernment', 'name': 'W...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,...,0,en,NaN,0.0,0.30,negative,0.079,0.921,0.000,-0.4404
3,2022-03-14 14:11:05-04:00,1503433550292525059,1503433550292525059,@DrEricDing Covid19 #omicron #chinesevirus _ P...,@drericding covid19 omicron chinesevirus _ per...,False,"[{'text': 'omicron', 'indices': [20, 28]}, {'t...","[{'screen_name': 'DrEricDing', 'name': 'Eric F...","<a href=""https://mobile.twitter.com"" rel=""nofo...",Sussex,...,0,en,NaN,0.0,0.00,positive,0.000,0.913,0.087,0.2732
4,2022-03-14 05:12:37-04:00,1503298039452258304,1503298039452258304,@Sloganyo @ETSU_MBB Indeed. That was a great ...,@sloganyo @etsu_mbb indeed. that was a great ...,False,"[{'text': 'ChineseVirus', 'indices': [53, 66]}]","[{'screen_name': 'Sloganyo', 'name': 'Logan Ha...","<a href=""https://mobile.twitter.com"" rel=""nofo...","Knoxville, TN",...,0,en,NaN,0.8,0.75,positive,0.000,0.661,0.339,0.6249


## Putting Everything Together

So far we've been experimental, working and debugging through code with a small sample size. Let's scrape 10,000 tweets and rerun this whole process.

In [2]:
###########################################
# Setup
###########################################

# importing dependencies
import tweepy as tw
import pandas as pd

# Twitter Keys, tokens, and secrets are saved in seperate config file on my local device
from config import consumer_key, consumer_secret, access_token, access_secret, bearer_token

# authenticate
auth = tw.OAuthHandler(consumer_key, consumer_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
###########################################
# Getting Tweets
###########################################

# get tweets from the API
search_query = "#covid19 -filter:retweets"

# center of united states
latitude = "37.09024"
longitude = "-95.712891"
radius = "791mi"
location = f"{latitude},{longitude},{radius}"
num_of_tweets = 10000

tweets = tw.Cursor(api.search_tweets,
                  q = search_query,
                  lang = "en",
                  geocode = location).items(num_of_tweets) # getting 10,000 tweets

# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print(f"{num_of_tweets} tweets have been scraped.")

###########################################
# Saving scraped data as a DataFrame object
###########################################
    
# Lets create a dataframe for our scraped tweets
df = pd.DataFrame(columns = tweets_copy[0]._json.keys())

for i in range(len(tweets_copy)):
    df = df.append(pd.json_normalize(tweets_copy[i]._json))
    
list_of_features_to_keep = ['created_at', 'id', 'id_str', 'text', 'truncated', 'entities.hashtags', 'entities.user_mentions',
                            'source', 'user.location', 'user.id', 'user.id_str', 'user.name',
                            'user.screen_name', 'user.description', 'user.url', 
                            'coordinates', 'place', 'contributors', 'is_quote_status', 'user.statuses_count',
                            'user.followers_count', 'user.friends_count', 'user.listed_count', 
                            'user.created_at', 'user.favourites_count', 'user.utc_offset', 'user.time_zone',
                            'retweet_count', 'favorite_count', 'lang', 'possibly_sensitive'
                            ]
df = df[list_of_features_to_keep]

# reset index
df = df.reset_index()
del df['index']

print(f"DataFrame has been created.")

###########################################
# Cleaning DataFrame
###########################################

# getting full tweet text instead of truncated tweet text
for i in range(len(df)):
    # only do this for those tweets that are truncated
    if df.at[i, 'truncated'] == True:   
        # get ID of the tweet
        id_str = list(df['id_str'])[i]
        
        # verify API status and get tweet info
        status = api.get_status(id_str, tweet_mode="extended")
        
        # replace truncated text with full text
        try:
            df['text'][i] = status.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            df['text'][i] = status.full_text
            
        # set the value of truncated at index i to False
        df.at[i, 'truncated'] = False
        
print("All tweets have been de-truncated.")
        
# clean text using regex (remove hyperlines, punctuation and convert to lowercase)
import re

tweets_cleaned = []

for tweet in df['text']:
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.\S+', "", tweet)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # convert to lower case
    tweet = tweet.lower()
    
    # append tweet text to list of cleaned tweet texts
    tweets_cleaned += [tweet]

# convert list to Pandas Series object
tweets_cleaned_series = pd.Series(tweets_cleaned)

# Add text_cleaned as column to df
df.insert(df.columns.get_loc("text") + 1, "text_cleaned", tweets_cleaned_series)

print("All tweets have been cleaned.")


# Convert timestamp from UTC to EST
from pytz import timezone
from datetime import datetime

def utc_to_est(tweet_timestamp):
        '''
        Utility function to convert timestamps from UTC to EST.
        '''
        
        eastern = timezone('US/Eastern')
        utc = timezone('UTC')
        created_at = datetime.strptime(tweet_timestamp, '%a %b %d %H:%M:%S +0000 %Y')
        utc_created_at = utc.localize(created_at)
        est_created_at = utc_created_at.astimezone(eastern)
        
        return est_created_at
    
# tweet created at
for i in range(len(df)):
    df['created_at'][i] = utc_to_est(df['created_at'][i])
    
# user account created at
for i in range(len(df)):
    df['user.created_at'][i] = utc_to_est(df['user.created_at'][i])
    
print("Timestamps have been converted to EST.")
    
###########################################
# Performing Sentiment Analysis
# - Generating new columns for df
###########################################
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# calulcating polarity and subjectivity for all tweets
df[['polarity', 'subjectivity']] = df['text_cleaned'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

for index, row in df['text_cleaned'].iteritems():
    # using Vader to get sentiment scores
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    
    # determine sentiment
    if neg > pos:
        df.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        df.loc[index, 'sentiment'] = "positive"
    else:
        df.loc[index, 'sentiment'] = "neutral"
    
    # populating neg, neutral, positive, and compound columns
    df.loc[index, 'neg'] = neg
    df.loc[index, 'neu'] = neu
    df.loc[index, 'pos'] = pos
    df.loc[index, 'compound'] = comp
    
print("Sentiment Analysis Complete.")